The **first code block** imports all the necessary packages. 
The **second code block** mounts google drive so that this script can read files located there.
The **third code block** ensures that we are reading from the correct directory.


In [1]:
from bs4 import BeautifulSoup
import os
import re
!pip install text2emotion
!pip install emoji==1.7
import text2emotion as te
import json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=387481132fa5b7353d6ec033505f27db85e537e816237bdd768fb9198cee1696
  Stored in directory: /root/.cache/pip/wheels/9a/b8/0f/f580817231cbf59f6ade9fd132ff60ada1de9f7dc85521f857
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 KB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171048 sha256=f25ba3053296232e749bec9134019a57290dbacdcaf61a9df6de03680b349de6
  Stored in directory: /root/.cache/pip/wheels/f

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
os.chdir('/content/drive/MyDrive/HTMLFiles/NoiseHTML')
directory = os.getcwd()

The **following code block** is used to explore the text. It is determined that **discussion items is an h2 tag**. p tags within the discussion items h2 tag 
contain the text that we want to extract.

In [6]:
with open('Meeting_Notes_2009_9_29.html') as file:
  soup = BeautifulSoup(file.read(), 'html.parser')
counter = 0
# find all li & h2 elements elements in the HTML
find_list_items = soup.find_all('li')
find_h1_tags = soup.find_all('h1')
find_pre_elements = soup.find_all('pre')

discussion_items_text = []

for h1 in soup.find_all('h1'):
  if 'Meeting Notes' in h1.text or 'Notes' in h1.text:
    for pre in h1.find_all_next('pre'):
      discussion_items_text.append(pre.text)
      print(pre.text)
      print('WHAT ARE YOU SAYING')
      for li in h1.find_all_next('li'):
        discussion_items_text.append(li.text)

print(discussion_items_text)

 good money situation
 some people haven't yet paid dues, explanation of dues payment, etc
 3 months of operating expenses in the bank, not minus pending build out expenses
 donation instructions

WHAT ARE YOU SAYING
 Electrical
    PG&E will turn on road side panel on next monday
    Power run to more locations in the space and getting rid of the extension cords
    Interface with individual space owners to setup space specific electrical requirements
    Tag wanted outlet/switch locations
    Rubin owns central table columns
    David owns chill space
    Crustcher and Jason D. own classrooms
    JSharp, Lamont, and Jim own library
 Housekeeping
    Trash cans replaced with larger units
    Shop trash can will not be picked up, trash needs to be removed
    Dr. Jesus loaned van to take things to dump
    To take garbage out: Monday night, curb -- Do not take out any recepticles that are not disposal company labeled
    Compact/Crush cans
    e-waste:
      scheduled pickup of e-waste

In the following code block we are iterating through all the meeting notes one by one, saving the document name and extracting the discussion text. The html header for the discussion text varies quite a bit from document to document, so we have included a series of if statements that seek to capture the discussion text regardless of the header type. There are however, still 28 documents for which the meeting notes have not been captured. A preliminary QA shows that most of these notes have very little information. 

In [7]:
document_name = [] 
text_list = []

for file in os.listdir(directory):
    with open(file, 'r') as fileHandle:
      mht_doc = fileHandle.read()
      soup = BeautifulSoup(mht_doc, 'html.parser')
      document_name.append((os.path.splitext(file)[0]))
      text = []
      for h1 in soup.find_all('h1'):
        if 'Discussion Items' in h1.text or 'Discussion Notes' in h1.text or 'Notes' in h1.text:
          for p in h1.find_all_next('p'):
            text.append(p.text)
          for pre in h1.find_all_next('pre'):
            text.append(pre.text)
          for li in h1.find_all_next('li'):
            text.append(li.text)
          for ul in h1.find_all_next('ul'):
            text.append(ul.text)
      for h2 in soup.find_all('h2'):
        if 'Meeting Notes' in h2.text or 'Notes' in h2.text or 'Discussion topics' in h2.text:
          for p in h2.find_all_next('p'):
              text.append(p.text)
          for li in h2.find_all_next('li'):
            text.append(li.text)
        elif 'Discussion Items' in h2.text:
          for p in h2.find_all_next('p'):
            text.append(p.text)
          for li in h2.find_all_next('li'):
            text.append(li.text)
      for h3 in soup.find_all('h3'):
        if 'Discussion Items' in h3.text:
          for p in h3.find_all_next('p'):
            text.append(p.text)
          for li in h3.find_all_next('li'):
            text.append(li.text)
        for b in soup.find_all('b'):
          if b.text == 'Agenda':
            for p in b.find_all_next('p'):
              text.append(p.text)
      text_list.append(text)
counter = 0
for i in range(len(text_list)):
  if len(text_list[i]) == 0:
    print(document_name[i])
    counter += 1

print(counter)

0


In [8]:
for doc, text in zip(document_name, text_list):
  text_clean = []
  for line in text:
    line = line.split(' ')
    tempList = []
    for word in line:
      if '=' not in word:
        tempList.append(word)
    line = ' '.join(tempList)
    text_clean.append(line)

In [9]:
dictionaryOfNotes = {}
for doc, text in zip(document_name, text_list):
    text_clean = []
    for line in text:
      line = line.split(' ')
      tempList = []
      for word in line:
        if '=' not in word:
          tempList.append(word)
      line = ' '.join(tempList)
      text_clean.append(line)
    dictionaryOfNotes[doc[0]] = '\n'.join(text_clean)

print(dictionaryOfNotes)


{'M': '\n\n8pm, place:  Noisebridge, 83C Wiese St. near 16th St. & Mission St., San Francisco\n\nFor project info see Hologlyphic Video Art.\n\nA while ago I bought the domainname digitalstorms.com, i figured it would be nice to do some sort of project related to info maps, and i think one of the first maps, might be one that blends economic data with blogs.  "we feel fine" is a great project that blends art, with sensory computing, and allows the web to emote just a little bit about us.\n\nthe idea here was to capture interesting data from a speech or conversation and somehow relate that to google searches and some sort of UI, either for art, fun, or profit.\n\n1 Time and Place\n2 Agenda\n3 Notes\n\n3.1 Hologlyphic Movie system, with possible funding\n\n\n3.1 Hologlyphic Movie system, with possible funding\n4 discussed projects\n\n4.1 Digital Storms\n4.2 voice stream\n\n\n4.1 Digital Storms\n4.2 voice stream\nQuick update on current running projects, past and upcoming events (to make 

In [11]:
 #print(te.get_emotion(' '.join(text_list[30])))

# te.get_emotion("wow this package is really amazing")
sentiment_dictionary = {}
for meeting, text in zip(document_name, text_list):
  sentiment_dictionary[meeting] = te.get_emotion(" ".join(text))


In [12]:
sentiment_dictionary2 = {}
for key, val in sentiment_dictionary.items():
  summ = 0
  for score in val.values():
    summ += score
  if summ != 0:
    sentiment_dictionary2[key] = val

Exporting data as a text file

In [13]:
os.getcwd()
with open('/content/drive/MyDrive/PracticumIIsentimentResults.txt','w') as fileHandle:
  json.dump(sentiment_dictionary2, fileHandle)

Writing code to put the data into a csv

In [14]:
name = []
Happy = []
Angry = []
Surprise = []
Sad = []
Fear = [] 
sentiments = ['Happy','Angry','Surprise','Sad','Fear']
for key, value in sentiment_dictionary2.items():
  name.append(key)
  for key2, value2 in value.items():
    for emotion in sentiments:
      if key2 == 'Happy':
         Happy.append(value2)
      elif key2 == 'Angry':
         Angry.append(value2)
      elif key2 == 'Surprise':
         Surprise.append(value2)
      elif key2 == 'Sad':
         Surprise.append(value2)
      elif key2 == 'Fear':
         Fear.append(value2)

print(len(Happy))
#for i in range(len(Happy)):
  #summm = Happy[i] + Angry[i] + Surprise[i] + Sad[i] + Fear[i]
  #print(summm)


3650


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = [7.50, 3.50]
plt.rcParams["figure.autolayout"] = True

headers = ['Meeting Dates', 'Happy', 'Angry','Surprise', 'Sad', 'Fear']

df = pd.read_csv('NB_Sentiments.csv', names=headers)

df.set_index('Name').plot()

plt.show()